In [1]:
# Suprise 패키지 설치
#pip install scikit-surprise

## Import

In [2]:
import pandas as pd
from surprise import SVD, Dataset, Reader, accuracy

## Data Load

In [3]:
train_df = pd.read_csv('_open/train.csv')
test_df = pd.read_csv('_open/test.csv')

In [4]:
# Surprise 라이브러리용 Reader 및 Dataset 객체 생성
reader = Reader(rating_scale=(0, 10))
train = Dataset.load_from_df(train_df[['User-ID', 'Book-ID', 'Book-Rating']], reader)
train = train.build_full_trainset()

## SVD

In [5]:
# SVD 모델 훈련
model = SVD()
model.fit(train)

## Prediction

In [6]:
submit = pd.read_csv('_open/sample_submission.csv')

In [7]:
submit['Book-Rating'] = test_df.apply(lambda row: model.predict(row['User-ID'], row['Book-ID']).est, axis=1)

## Submission

In [8]:
submit.to_csv('results/baseline_submit.csv', index=False)

# Evaluation

In [9]:
from sklearn.metrics import mean_squared_error

In [10]:
train_pred = train_df.apply(lambda row: model.predict(row['User-ID'], row['Book-ID']).est, axis=1)
rmse = mean_squared_error(train_df['Book-Rating'], train_pred, squared=False)
rmse

1.0398539062176348

train rmse = 1.03985

# 데이콘 의견

본 노트북은 데이콘에서 제공하는 베이스라인으로 제출을 목표로 함. SVD를 활용하여 예측을 수행하는데 평가 데이터 셋에 존재하는 유저나 도서가 학습 데이터 세트에 없을 수 있어 SVD는 적절하지 않을 수 있다고 경고 (Cold Start Problem)

# 나의 의견

여느 CF와 마찬가지로 유저id, 책id 만 가지고 평점을 학습하기 때문에 당연히 cold start 문제가 발생함. 오류를 야기하지는 않으나 학습되지 않은 콘텐츠와 유저에 대해서 그저 id의 순서에 따라서 평점을 예측하게 되므로 적절하지 않은 예측임에 분명함. 이를 고려할 수 있는 모델을 만들 필요가 있음